# Análise exploratória de dados

**Abaixo se encontra a importação das bibliotecas e carregamento dos dados**

Anteriormente, a limpeza dos dados foi realizada, resultando no arquivo 'ENEM_CO_TRATADO.parquet'.
Este arquivo será o foco de nossas análises.


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import numpy as np
import scipy.stats as stats
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px

dados_tratados = pd.read_parquet('ENEM_CO_TRATADO.parquet')

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


**Tabelas "de-para"** são estruturas fundamentais em análise de dados e processamento de informações. Elas permitem a criação de associações entre valores originais e valores correspondentes, simplificando a interpretação e manipulação de dados complexos. Com funções específicas para cada tabela, como idade, cor/raça, nacionalidade, situação de conclusão, ano de conclusão e status de redação, podemos modularizar a criação dessas tabelas, tornando o código mais organizado e fácil de manter. Assim, o arquivo central pode importar essas funções prontas para uso, economizando tempo e facilitando a manipulação de dados em projetos analíticos. Essas tabelas são uma ferramenta valiosa para simplificar dados complexos e obter insights significativos de forma eficiente.

In [2]:
from tabelas_de_para import (
    criar_tabela_de_para_idades,
    criar_tabela_de_para_cor_raca,
    criar_tabela_de_para_nacionalidade,
    criar_tabela_de_para_situacao_conclusao,
    criar_tabela_de_para_ano_conclusao,
    criar_tabela_de_para_status_redacao
)


In [3]:
tabela_idades = criar_tabela_de_para_idades()
tabela_cor_raca = criar_tabela_de_para_cor_raca()
tabela_nacionalidade = criar_tabela_de_para_nacionalidade()
tabela_situacao_conclusao = criar_tabela_de_para_situacao_conclusao()
tabela_ano_conclusao = criar_tabela_de_para_ano_conclusao()
tabela_status_redacao = criar_tabela_de_para_status_redacao()